In [35]:
from math import floor, log
from collections import defaultdict
import random
import tqdm
import numpy as np
from numpy.random import poisson, binomial, uniform, exponential, multinomial, normal, multivariate_normal
from scipy.stats import norm
from scipy.stats import kstest, norm

def dice():
    return random.randint(1, 6)
ct = 10000

def multi(dist):
    v = multinomial(1, dist)
    v = np.where(v == 1)[0][0]
    return v

def markov_start(start, P, length):
    '''
    Runs Markov Chain with a starting distribution, generates {length} items. 
    '''
    cur = start
    seq = []
    for i in range(length):
        distribution = P[cur].flatten()
        v = multinomial(1, distribution)
        v = np.where(v == 1)[0][0]
        seq.append(v)
        cur = v
    return seq

def markov_start_absorb(start, P, absorb_set):
    '''
    Runs Markov Chain with a starting distribution, generates {length} items. 
    '''
    cur = start
    seq = []
    while True:
        distribution = P[cur].flatten()
        v = multi(distribution)
        seq.append(v)
        cur = v
        if cur in absorb_set:
            break
    return seq

def markov_rdm(dist, P, length):
    '''
    Runs Markov Chain with a starting distribution, generates {length} + 1 items. 
    '''
    cur = np.nonzero(multinomial(1, dist))[0]
    seq = [cur]
    for i in range(length):
        distribution = P[cur].flatten()
        v = np.nonzero(multinomial(1, distribution))[0]
        seq.append(v)
        cur = v
    return seq

In [14]:
def q1():
    dist = [0.1, 0.3, 0.2, 0.4]
    mat = np.zeros((4, 4))
    for i in range(1000):
        best = 0
        for j in range(200):
            prev = best
            best = max(best, multi(dist))
            mat[prev, best] += 1
    row_sums = np.sum(mat, axis = 1).reshape((4, 1))
    print(mat / row_sums)

q1()

[[0.0990991  0.30540541 0.2018018  0.39369369]
 [0.         0.4302521  0.19663866 0.37310924]
 [0.         0.         0.57428215 0.42571785]
 [0.         0.         0.         1.        ]]


In [34]:
def q2i():
    alpha = np.random.random()
    P_0 = np.random.random()
    initial_dist = [P_0, 1 - P_0]
    print(f"alpha: {alpha}, initial_dist: {initial_dist}")

    P = np.array([[alpha, 1 - alpha], [1 - alpha, alpha]])

    success = 0
    for i in tqdm.tqdm(range(ct)):
        start = multi(initial_dist)
        chain = markov_start(start, P, 3)
        if start == chain[2]:
            success += 1
    print(f"Empirical: {success / ct}")
    print(f"Analytical: {alpha ** 3 + 3 * (1 - alpha) ** 2 * alpha}")
for i in range(5):
    print(f"=== Trial {i} ===")
    q2i()

    

=== Trial 0 ===
alpha: 0.7369670786503182, initial_dist: [0.7265213319661465, 0.27347866803385346]


100%|██████████| 10000/10000 [00:00<00:00, 16077.26it/s]


Empirical: 0.5569
Analytical: 0.5532260251707446
=== Trial 1 ===
alpha: 0.8123860587355617, initial_dist: [0.10036704695407495, 0.899632953045925]


100%|██████████| 10000/10000 [00:00<00:00, 16528.91it/s]


Empirical: 0.6154
Analytical: 0.6219368362596134
=== Trial 2 ===
alpha: 0.0561603057874277, initial_dist: [0.996267166035717, 0.0037328339642830333]


...

Empirical: 0.1547
Analytical: 0.15026555159834876
=== Trial 3 ===
alpha: 0.28461132193349203, initial_dist: [0.6118976514014131, 0.38810234859858694]


100%|██████████| 10000/10000 [00:00<00:00, 16750.51it/s]


Empirical: 0.4574
Analytical: 0.460030510279387
=== Trial 4 ===
alpha: 0.6989141315579425, initial_dist: [0.7725125698918722, 0.22748743010812777]


100%|██████████| 10000/10000 [00:00<00:00, 16474.51it/s]

Empirical: 0.5364
Analytical: 0.5314816078910342


In [33]:
def q2ii():
    N = np.random.randint(3, 5)
    alpha = np.random.random()
    P_0 = np.random.random()
    initial_dist = [P_0, 1 - P_0]
    print(f"alpha: {alpha}, initial_dist: {initial_dist}, N: {N}")

    P = np.array([[alpha, 1 - alpha], [1 - alpha, alpha]])

    success = 0
    for i in tqdm.tqdm(range(ct)):
        start = multi(initial_dist)
        chain = markov_start(start, P, N + 1)
        s_chain = sum(chain)
        if s_chain == 0 or s_chain == N + 1:
            success += 1
    print(f"Empirical: {success / ct}")
    print(f"Analytical: {alpha ** N}")
for i in range(5):
    print(f"=== Trial {i} ===")
    q2ii()

=== Trial 0 ===
alpha: 0.9712301930155816, initial_dist: [0.997848159474362, 0.0021518405256379936], N: 4


100%|██████████| 10000/10000 [00:00<00:00, 10303.96it/s]


Empirical: 0.8942
Analytical: 0.889792416632703
=== Trial 1 ===
alpha: 0.5816266507340696, initial_dist: [0.030509287603755042, 0.969490712396245], N: 3


100%|██████████| 10000/10000 [00:00<00:00, 12650.29it/s]


Empirical: 0.1986
Analytical: 0.1967582242520714
=== Trial 2 ===
alpha: 0.23005530370936134, initial_dist: [0.2427146140692188, 0.7572853859307812], N: 4


...

Empirical: 0.0041
Analytical: 0.002801102491850807
=== Trial 3 ===
alpha: 0.21548827495809142, initial_dist: [0.37822180924689486, 0.6217781907531051], N: 3


100%|██████████| 10000/10000 [00:00<00:00, 12903.28it/s]


Empirical: 0.0097
Analytical: 0.01000624042224453
=== Trial 4 ===
alpha: 0.7345172115084547, initial_dist: [0.6860249074983729, 0.31397509250162714], N: 3


100%|██████████| 10000/10000 [00:00<00:00, 13046.32it/s]

Empirical: 0.3995
Analytical: 0.3962834456007585


In [39]:
def q3():
    P = np.array([[1, 0, 0], [0.1, 0.6, 0.3], [0, 0, 1]])

    success = 0
    absorb = set([0, 2])
    chain_sum = 0
    for i in tqdm.tqdm(range(ct)):
        chain = markov_start_absorb(1, P, absorb)
        if chain[-1] == 0:
            success += 1
        chain_sum += len(chain)
    print(f"Empirical Success: {success / ct}")
    print(f"Analytical Success: 0.25")
    print("=== === ===")
    print(f"Empirical Mean Time: {chain_sum / ct}")
    print(f"Analytical Success: 2.5")
q3()

100%|██████████| 10000/10000 [00:00<00:00, 25510.22it/s]

Empirical Success: 0.2472
Analytical Success: 0.25
=== === ===
Empirical Mean Time: 2.4912
Analytical Success: 2.5


In [44]:
def q4():
    P = np.array([[1, 0, 0, 0], [0.1, 0.2, 0.5, 0.2], [0.1, 0.2, 0.6, 0.1], [0.2, 0.2, 0.3, 0.3]])
    
    absorb = set([0])
    success = 0
    for i in tqdm.tqdm(range(ct)):
        chain = markov_start_absorb(2, P, absorb)
        if all(f != 1 for f in chain):
            success += 1

    print(f"Empirical: {success / ct}")
    print(f"Analyticall: 0.36")
q4()

100%|██████████| 10000/10000 [00:01<00:00, 7385.57it/s]

Empirical: 0.367
Analyticall: 0.36


In [51]:
def q5():
    P = np.array([[1, 0, 0, 0], [0.1, 0.3, 0.5, 0.1], [0.2, 0.1, 0.6, 0.1], [0, 0, 0, 1]])
    # Q = 1 2
    Q = np.array([[0.3, 0.5], [0.1, 0.6]])
    print(np.linalg.inv(np.eye(2) - Q) @ np.ones((2, 1)))
    absorb = set([0, 3])
    sum_len = 0
    for i in tqdm.tqdm(range(ct)):
        chain = markov_start_absorb(1, P, absorb)
        sum_len += len(chain)
    print(f"Empirical: {sum_len / ct}")
    print(f"Analytical: 3.912")
q5()

[[3.91304348]
 [3.47826087]]


100%|██████████| 10000/10000 [00:00<00:00, 15698.59it/s]

Empirical: 3.913
Analytical: 3.912


In [53]:
def q5():
    P = np.array([[1, 0, 0, 0], [0.1, 0.3, 0.5, 0.1], [0.2, 0.1, 0.6, 0.1], [0, 0, 0, 1]])
    # Q = 1 2
    absorb = set([0, 3])
    time_1, time_2 = 0, 0
    for i in tqdm.tqdm(range(ct)):
        chain = [1] + markov_start_absorb(1, P, absorb)
        time_1 += sum([c == 1 for c in chain])
        time_2 += sum([c == 2 for c in chain])
    time_1, time_2 = time_1 / ct, time_2 / ct
    print(f"Empirical: time_1: {time_1}, time_2: {time_2}, sum: {time_1 + time_2}")
    print(f"Analytical: 1.739 + 2.174 = 3.912")
q5()

100%|██████████| 10000/10000 [00:00<00:00, 11961.73it/s]

Empirical: time_1: 1.7358, time_2: 2.17, sum: 3.9058
Analytical: 1.739 + 2.174 = 3.912
